In [2]:
import os, json
import string
import re
import pandas as pd

# 1. Helper Functions

Following codes define functions that are needed to load/parse Facebook data files, and transform the data.

* `parse_json`: Read json files from the path. `file_end` argument is used to filter out the mistakenly submitted files.
* `name_normal`: Normalize keywords in the data. (a) de-captitalize keywords, (b) delete punctuations, and (c) delete a single word initial in the middle (to delete middle names).
* `expand_to_df`: Expand a dictionary that contains each R's data (key: user id / value: list of keywords from the data) into a regular Pandas data frame.

IMPORTANT: `name_normal` is not applied to any of following codes to ensure the precision of the matching process. Should be applied later.

In [17]:
def parse_json(path, file_end):
    path = path + '/'
    json_files = [pos_json for pos_json in os.listdir(path) if pos_json.endswith(file_end)]
    
    out = dict()
    
    for file in json_files[0:5]:
        user_id = re.sub(file_end + "$", "", file)  # Filter files with a pariticular file end (e.g. "_pages.json")
        full_file = path + file 
        
        try:
            with open(full_file, 'r', encoding='UTF-8') as f:
                out[user_id] = json.load(f)
        except:
            continue
            
    return out

translator=str.maketrans('','',string.punctuation)

def name_normal(name):
    out = name.lower().translate(translator)  # (a) De-capitalize, and (b) delete punctuations.
    out = re.sub("\s[A-Za-z]\s", " ", out)    # (c) Delete single characters in the middle.
    " ".join(out.split())
    
    return out

def expand_to_df(matched_keywords):
    isin_keywords = dict()
    
    potent_keywords = list(set().union(*matched_keywords.values())) # Get all the keywords that appeared in the data at least once.
    
    for key, keywords in matched_keywords.items():
        isin_keywords[key] = [item in keywords for item in potent_keywords] # Expand a list of keywords into a seriese of TRUE/FALSE.
        
    df = pd.DataFrame.from_dict(isin_keywords, orient='index', columns = potent_keywords)
    
    return df

# 2. Load Data
## 2.1. Load and Parse JSON Files

In [4]:
dt_json = parse_json("data/Qualtrics panel data/Q50", "_pages.json")

Following code transforms the loaded liked pages to a dictionary with keys being user id's and values being lists of liked pages.

In [5]:
Index = dict()

for key, value in dt_json.items():
    Index[key] = list()
    if 'page_likes' in value.keys():
        test_key = value['page_likes'][0].keys()
        
        if 'data' in test_key:        # This if clause is needed to load two different formats of JSON data.
            for item in value['page_likes']:
                Index[key].append(item['data'][0]['name'])
        else:
            for item in value['page_likes']:
                Index[key].append(item['name'])
    else:
        continue

In [6]:
dt_ads = parse_json("data/Qualtrics panel data/Q52", "_ads_interests.json")

Following code transforms the loaded advertising categories to a dictionary with keys being user id's and values being lists of advertising categories.

In [7]:
ads_topics = dict()

for key, value in dt_ads.items():
    ads_topics[key] = list()
    
    try:
        if 'topics' in value.keys():
            for item in value['topics']:
                ads_topics[key].append(item)
        else:
            continue
    except:
        continue

## 2.2. Load Dictionary

In [8]:
# Takes about a minute.
dic = pd.read_excel("Civic customization Dictionary 02-10-19.xlsx")

Transform the loaded MS Excel filte to a long list of keywords. 

In [9]:
dic = dic['Full Name'].tolist()

# 3. Matching
## 3.1.  Perfect Matching
Perfect matching simply converts two lists of keywords (a dictionary & R's keywords) into sets, and obtain intersection. The outcome is a dictionary with keys being user id and values being matched keywords.

In [10]:
def perfect_matching(dict_keyword, dic):
    matched_keywords = dict()
    dic = set(dic)  # Convert a dictionary to a set
    
    for key, value in dict_keyword.items():
        
        matched_keywords[key] = list(set(value) & dic) # (a) Convert a list of keywords into a set, and (b) Get the intersection. 
    
    return matched_keywords

In [12]:
Index_perfect = perfect_matching(Index, dic)

In [13]:
ads_topics_perfect = perfect_matching(ads_topics, dic)

In [18]:
df_Index_perfect = expand_to_df(Index_perfect)

In [20]:
df_Index_perfect

,Cheech and Chong,Office for Businesses,The Mind Unleashed,Seventh Generation,Occupy Democrats,Captain D's,Love is a Rainbow,The Boeing Company,Jewlish,Michigan Lottery,...,Saranoni Luxury Blankets,Barack Obama,Metallica,JWOWW,Avocado Green Mattress,Hawaiian Airlines,Target,Al Jazeera English,FOX Michiana,ShopLivin.com
R_08Owi7R4nr4tQo9,True,False,False,True,False,True,True,False,False,True,...,True,True,False,True,True,False,True,False,True,True
R_11haqA47J6Xz6uy,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
R_0kVxSmqT4eW338t,False,True,True,False,True,False,False,False,False,False,...,False,False,True,False,False,False,False,True,False,False
R_10uxpY89ANp6Lpk,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
R_10vnVW9jokF0wMQ,False,False,False,False,False,False,False,True,True,False,...,False,False,False,False,False,True,False,False,False,False


In [21]:
df_ads_topics_perfect = expand_to_df(ads_topics_perfect)

In [22]:
df_ads_topics_perfect

,Scribd,Literature,Friendship,A&W Restaurants,Reddit,U.S. state,Blue Apron,Warren G. Harding,Vacations,McDonald's,...,Train,Feminism,County seat,Whisper,Female,NBC,People,T.I.,Barack Obama,WNDU-TV
R_0IgALa5sTzbWuDT,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
R_0kVxSmqT4eW338t,True,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
R_10uxpY89ANp6Lpk,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
R_08Owi7R4nr4tQo9,False,True,True,False,False,True,True,True,True,False,...,True,True,True,False,True,False,True,False,True,True
R_10SJJi1eOYSQQGR,False,True,True,True,False,True,False,False,False,True,...,False,False,True,True,True,True,True,True,False,False


## 3.2.  Fuzzy Matching
Fuzzy matching is harder: (a) It breaks every keyword and dictionary entry into a list of words (e.g. "Donald Trump" -> ["Donald", "Trump"]), and (b) if the list of words is a subset of any list of words in the dictionary (e.g. a keyword in the data: ["Donald", "Trump"] <= an entry from the dictionary: ["President", "Donald", "Trump"]), then the keyword is considered as matched.  

This process cannot entertain the efficient Python set operation to match anymore. This algorithm simply compare every single keyword in the data with every single entry in the dictionary sequentially. Suppose that 300 respondents have 500 keywords on average, and that the dictionary has 160,000 entries. Then the opertation is conduected 300 x 500 x 160,000 = 24 billion times. This is definitely where parallelizing can help reduce the computing time, but it hasn't been implemented yet.

In [51]:
def fuzzy_matching(dict_keyword, dic):
    matched_keywords = dict()
    
    for key, value in dict_keyword.items():  # For each respondent
        matched_keywords[key] = list()
        for liked_word in value:             # For each keyword in the data
            for dic_word in dic:             # For each entry in the dictionary
                if set(str(liked_word).split()) <= set(str(dic_word).split()):   # If a splitted keyword is a subset of a splitted entry  
                    matched_keywords[key].append(liked_word)  # Then add the keyword in the matched keyword list
                    break                                     # If the keyword was matched, stop matching with that keyword
    
    return matched_keywords

In [52]:
%%time
fuzzy_matching(Index, dic)

Wall time: 7min 33s


{'R_08Owi7R4nr4tQo9': ["Zaxby's",
  'i play., Inc.',
  'Blue Apron',
  'Saranoni Luxury Blankets',
  'Love is a Rainbow',
  'Tidy Tots Diapers',
  'Equinox International',
  'Winnie',
  'Elevated',
  'Sinkboss',
  'Bumbleride',
  'Avocado Green Mattress',
  'The Penny Hoarder',
  'ShopLivin.com',
  'Cosmopolitan',
  'Bella Luna',
  'Casper',
  'Casper',
  'theSkimm',
  'EcoVessel',
  'Nielsen',
  'Racked',
  "Jack's",
  'Taco Bell',
  'Mama Strut by Pelv-Ice',
  "Captain D's",
  'Barack Obama',
  'FOX Michiana',
  'ShadeTree',
  'Michigan Lottery',
  'Starbucks',
  'Speedway',
  'Marie Claire',
  '1,000 Days',
  'SheaMoisture',
  'PureWow',
  'Extra',
  'Discount Tire',
  'Primary.com',
  'Shared',
  'Carlito',
  'The Walking Dead',
  'Cheech and Chong',
  'Buffalo Wild Wings',
  'Gay, Lesbian, Bisexual, Transgender, Straight Alliance',
  "Applebee's Grill + Bar",
  'Target',
  'Walmart',
  'Seventh Generation',
  'Taco Bell',
  'Hanes',
  'Tennessee',
  'JWOWW',
  'CSI'],
 'R_0kVxSmqT

In [48]:
Index

{'R_08Owi7R4nr4tQo9': ['Conor Maynard',
  'Center For Orthodontics',
  'Kiddington',
  'Little Acorn Preschool Learning Center',
  'Momma Be',
  'LulyBoo',
  'otteroo',
  'ro.sham.bo baby',
  'Forney Industries',
  'IL MAKIAGE',
  'Scouts 4031 Lexington, Tennessee',
  'Baby Brezza, USA',
  'Lucky Day',
  'Princess Theatre of Lexington, TN',
  "Zaxby's",
  'The Berry Clinic',
  'Supreme Giveaways and Contests',
  'Crunch-a-Mame Edamame Snacks',
  'Thegroomedman',
  'The Discoverer',
  'i play., Inc.',
  'Snappy Tomato Pizza - South Jackson',
  'Baby Memory Book',
  'Open Season Sports & Marine',
  "Lauren's Crafty Corner",
  "AJ's Baby & Toddler Store",
  'Get Rich or Dye Tryin',
  'Murfreesboro Sports Com',
  'Blue Apron',
  'Lil James',
  'Mum Luvs Brands Club',
  "Loretta Lynn's Ranch - Campground,  Tours, Concerts & Events",
  'Baby Benjamin',
  'Sybaris Pool Suites',
  'Carnival Cruises Fans',
  'Dirt Cheap',
  "Baby Merlin's Magic Sleepsuit",
  'Baja Grill',
  'Saranoni Luxury Bla